In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
	"NlpHUST/ner-vietnamese-electra-base", truncation=True, padding=False, max_length=256)
model = AutoModelForTokenClassification.from_pretrained(
	"NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device="cuda")
example = "Bước vào trận đấu, Barcelona nhanh chóng tràn lên tấn công nhưng vấp phải sự kháng cự quyết liệt của Sevilla. Bước ngoặt đến ở phút 22, Raphinha bị phạm lỗi trong vòng cấm và trọng tài đã cho Barcelona hưởng quả phạt đền. Trên chấm 11m, Lewandowski đã ghi bàn mở tỷ số cho Barcelona. Đến phút 28, Pedri đã có bàn nhân đôi cách biệt cho đội chủ nhà trước khi Lewandowski hoàn tất cú đúp cho riêng mình ở phút 39. Hiệp 1 khép lại với tỉ số 3-0 nghiêng về Barcelona. Sang hiệp 2, các chân sút của Barcelona liên tiếp bắn phá khung thành của Sevilla nhưng rất tiếc đều rơi vào thế việt vị. Tuy nhiên, chỉ trong vòng ít phút từ phút 82 đến 88, các khán giả có mặt trên sân Olimpic Lluis Companys đã được chứng kiến tới 3 bàn thắng. Cầu thủ vào sân thay người bên phía Barcelona là Torre lập cú đúp, xen giữa là bàn thắng danh dự của Idumbo bên phía Sevilla. Chung cuộc, Barca thắng Sevilal với tỷ số 5-1. Như vậy, đoàn quân của HLV Hansi Flick củng cố vị trí dẫn đầu trên bảng xếp hạng La Liga, duy trì khoảng cách 3 điểm với đại kình địch Real Madrid. "

ner_results = nlp(example)
print(ner_results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORGANIZATION', 'score': 0.99933404, 'index': 6, 'word': 'Barcelona', 'start': 19, 'end': 28}, {'entity': 'B-ORGANIZATION', 'score': 0.9992932, 'index': 22, 'word': 'Sevilla', 'start': 101, 'end': 108}, {'entity': 'B-PERSON', 'score': 0.99948126, 'index': 31, 'word': 'Rap', 'start': 136, 'end': 139}, {'entity': 'I-PERSON', 'score': 0.99870324, 'index': 32, 'word': '##hinh', 'start': 139, 'end': 143}, {'entity': 'I-PERSON', 'score': 0.9991743, 'index': 33, 'word': '##a', 'start': 143, 'end': 144}, {'entity': 'B-ORGANIZATION', 'score': 0.9993406, 'index': 45, 'word': 'Barcelona', 'start': 192, 'end': 201}, {'entity': 'B-PERSON', 'score': 0.99948275, 'index': 56, 'word': 'Lewandowski', 'start': 237, 'end': 248}, {'entity': 'B-ORGANIZATION', 'score': 0.9993654, 'index': 64, 'word': 'Barcelona', 'start': 273, 'end': 282}, {'entity': 'B-PERSON', 'score': 0.9984067, 'index': 70, 'word': 'Pe', 'start': 297, 'end': 299}, {'entity': 'I-PERSON', 'score': 0.998075, 'index': 71, 'word

In [2]:
# Sort ner_results by entry['start']
ner_results.sort(key=lambda x: x['start'])
print(ner_results)


def find_continous_keyword(ner_results):
	results = []
	cur = ""
	last_idx = -1
	for entry in ner_results:
		if (cur == ""):
			cur += entry['word']
			last_idx = entry['end']
		else:
			if (entry['start'] == (last_idx + 1)):
				cur += " " + entry['word']
				last_idx = entry['end']
			else:
				results.append(cur)
				cur = entry['word']
				last_idx = entry['end']
	results.append(cur)
	for i in results:
		if ('.' in i or len(i) < 2):
			results.remove(i)
	return results


def process_subword(ctn_kw: list):
	results = []
	for kw in ctn_kw:
		if ('#' not in kw):
			results.append(kw)
		else:
			if (len(results) == 0):
				results.append(kw.replace('#', ''))
			else:
				results[-1] += kw.replace('#', '')
	for i in results:
		if ('.' in i or len(i) < 2):
			results.remove(i)
	return set(results)


print(process_subword(find_continous_keyword(ner_results)))

[{'entity': 'B-ORGANIZATION', 'score': 0.99933404, 'index': 6, 'word': 'Barcelona', 'start': 19, 'end': 28}, {'entity': 'B-ORGANIZATION', 'score': 0.9992932, 'index': 22, 'word': 'Sevilla', 'start': 101, 'end': 108}, {'entity': 'B-PERSON', 'score': 0.99948126, 'index': 31, 'word': 'Rap', 'start': 136, 'end': 139}, {'entity': 'I-PERSON', 'score': 0.99870324, 'index': 32, 'word': '##hinh', 'start': 139, 'end': 143}, {'entity': 'I-PERSON', 'score': 0.9991743, 'index': 33, 'word': '##a', 'start': 143, 'end': 144}, {'entity': 'B-ORGANIZATION', 'score': 0.9993406, 'index': 45, 'word': 'Barcelona', 'start': 192, 'end': 201}, {'entity': 'B-PERSON', 'score': 0.99948275, 'index': 56, 'word': 'Lewandowski', 'start': 237, 'end': 248}, {'entity': 'B-ORGANIZATION', 'score': 0.9993654, 'index': 64, 'word': 'Barcelona', 'start': 273, 'end': 282}, {'entity': 'B-PERSON', 'score': 0.9984067, 'index': 70, 'word': 'Pe', 'start': 297, 'end': 299}, {'entity': 'I-PERSON', 'score': 0.998075, 'index': 71, 'word

In [3]:
def get_ner_results(text: str):
	ner_results = nlp(text)
	ner_results = [i for i in ner_results if i['score'] >= 0.95]
	ner_results.sort(key=lambda x: x['start'])
	return process_subword(find_continous_keyword(ner_results))

### Function to process long text

In [4]:
def process_long_text(text: str, tokenizer, max_length=510):
    # Check if text needs splitting
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    input_ids = tokens['input_ids'][0]

    if len(input_ids) <= max_length:
        return get_ner_results(text)

    # Split into sentences
    sentences = text.split('.')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]

    # Combine sentences into chunks
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        # Check length with new sentence
        test_text = ' '.join(current_chunk + [sentence])
        test_tokens = tokenizer(test_text, return_tensors="pt")
        test_length = len(test_tokens['input_ids'][0])
        if current_length <= max_length:
            current_chunk.append(sentence)
            current_length += test_length
        else:
            # Save current chunk and start new one
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_length = len(
                tokenizer(sentence, return_tensors="pt")['input_ids'][0])

    # Add final chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    # Process chunks and combine results
    all_entities = set()
    for chunk in chunks:
        chunk_entities = get_ner_results(chunk)
        all_entities.update(chunk_entities)

    return all_entities

In [5]:
print(process_long_text(example, tokenizer))

{'Sevilla', 'sân Olimpic Lluis Companys', 'Raphinha', 'Torredumbo', 'Pedri', 'Barca', 'Real Madrid', 'Hansi Flick', 'Sevilal', 'Lewandowski', 'Barcelona'}


### Read data

In [6]:
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
DATA_PATH = os.getenv("DATA_PATH")

data = pd.read_csv(DATA_PATH+"articles_training.tsv", sep='\t')
data

,content,tags
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ...","La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,..."
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ..."
2,Giải vô địch ná cao su thế giới năm 2024 đã di...,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng..."
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục..."
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...","toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề..."
...,...,...
99845,Tạo động lực phát triển nhanh và bền vững của ...,"Kỳ họp,Quốc hội,tư duy,Trần Thanh Mẫn,nguồn lự..."
99846,"Theo số liệu của Tổng cục Hải quan, trong 9 th...","cao su tự nhiên,cao su,ANRPC,Chứng khoán MB,nư..."
99847,Mong có chiến lược ứng phó hiệu quả với thiên ...,"Kỳ họp thứ Tám,Trần Thanh Mẫn,ngắn gọn,Chủ tịc..."
99848,"Nhà chức trách xác định, Hoàng Văn Thảo có hàn...","Hoàng Văn Thảo,chạy án,Cục phó,giả danh,TP Hồ ..."


In [7]:
# get longest text
longest_text = data['content'].apply(len).idxmax()
longest_text = data['content'][longest_text]
print(longest_text)

Đường về Xứ Phật - Tập 6 (Phần 3/3) LẤY ÂN BÁO OÁN LỜI PHẬT DẠY “Hận thù diệt hận thù Đời này không thể có Từ bi diệt hận thù Là định luật thiên thu”. “Và người khác không biết Chúng ta đây bị hại Chỗ ấy, ai hiểu được Tranh luận được lắng êm”. (Kinh Pháp Cú: I.Yamakavagga.Phẩm Song Yếu) CHÚ GIẢI: Bài kệ thứ năm hai câu đầu là một tà tư duy giúp cho chúng ta hiểu rõ nghĩa lý sống của cuộc đời: Không thể lấy hận thù diệt hận thù được. “Hận thù diệt hận thù Đời này không thể có” Lời dạy này là một chân lý không thể ai thay đổi được. Bởi vì cuộc đời này không thể lấy hận thù diệt hận thù được. Mà chỉ có lòng thương yêu của chúng ta mới diệt được hận thù. Hai câu dưới đây mới thật sự là Chánh tư duy: “Từ bi diệt hận thù Là định luật thiên thu” Sự tư duy này giúp cho chúng ta thấu rõ định luật bất di bất dịch của loài người, duy nhất chỉ có lòng thương yêu mới diệt được hận thù. Khi chúng ta tư duy suy nghĩ như vậy, giúp cho chúng ta lớn mạnh trong nền đạo đức nhân bản - nhân quả. Một bài kệ

In [8]:
print(process_long_text(longest_text, tokenizer))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'Cực', 'Bắc Tông', 'Diệu Quang', 'Đế', 'tu viện Chơn Như', 'kinh Duy Ma C', 'Duy Ma Cất', 'Thủy', 'Đại', 'Thế Tôn', 'Đại Ca Diếp', 'Mục Kiền Liên', 'giáo', 'Châu', 'TL', 'Phật Giáo Nguyên Thủy', 'Lạc', 'Thông', 'Niết Bàn', 'kinh Duy Ma', 'Thường Chiếu', 'Phật học', 'Thích Ca', 'Phật giáo Nguyên', 'Nhà xuất bản Tôn giáo', 'Nam Tông', 'Ca Diếp', 'Ấn Độ', 'Đề Bà Đạt Đa', 'Pháp Loa', 'Phật', 'TPHCM', 'Cực Lạc', 'Đế Thích', 'Tông', 'Ng', 'Tây', 'Phật giáo Nguyên Thủy', 'Thiền Đông Độ', 'A Nan', 'Nguyễn Hòa', 'Bà La Môn', 'Thông Lạc', 'Tuệ Hạnh', 'Minh Hoàng', 'Duy Ma', 'Độ', 'Võ Đế', 'Duy Ma C', 'Anan', 'giáo Nguyên', 'Hoa Sen', 'Nan', 'Thiên', 'Huệ Năng', 'Viễn', 'đạo Phật', 'Trung Hoa', 'Bồ Đề Đạt Ma', 'Đường về', 'Phật giáo', 'núi Linh Thú', 'Duy Ma Cật', 'Huyền Quang', 'Phật Thích Ca', 'Thư Viện Hoa Sen', 'Việt Nam', 'giáo Nguyên Thủy'}


In [9]:
# print the tags of the longest_text
print(data['tags'][data['content'] == longest_text].values[0].split(','))

['Pháp Cú', 'Phật', 'tiết độ', 'hận thù', 'nhẫn nhục', 'pháp môn', 'từ bi', 'câu kệ', 'đường về', 'biếng nhác', 'Tây Phương Cực Lạc', 'A La Hán', 'cảnh giới', 'chỗ ấy', 'A di đà kinh', 'Tam Minh', 'định luật', 'chứng quả', 'thế gian', 'thương yêu']


In [10]:
# f1 score
def f1_score_strings(pred, true):
	pred = set(pred)
	true = set(true)
	# normalize all strings
	pred = {i.lower() for i in pred}
	true = {i.lower() for i in true}
	tp = len(pred.intersection(true))
	fp = len(pred - true)
	fn = len(true - pred)
	if (tp + fp) == 0 or (tp + fn) == 0:
		return 0
	precision = tp / (tp + fp)
	recall = tp / (tp + fn)
	if (precision + recall) == 0:
		return 0
	f1 = 2 * (precision * recall) / (precision + recall)
	return f1

print(f1_score_strings(process_long_text(longest_text, tokenizer), data['tags'][data['content'] == longest_text].values[0].split(',')))

def accuracy_strings(pred, true):
	pred = set(pred)
	true = set(true)
	# normalize all strings
	pred = {i.lower() for i in pred}
	true = {i.lower() for i in true}
	return len(pred.intersection(true)) / len(true)

0.04597701149425287


In [11]:
f1_scores = []
accuracy_scores = []

for i in data.sample(n=100, random_state=1).index:
	pred = process_long_text(data['content'][i], tokenizer)
	true = data['tags'][i].split(',')
	f1 = f1_score_strings(pred, true)
	f1_scores.append(f1)
	accuracy = accuracy_strings(pred, true)
	accuracy_scores.append(accuracy)

# average f1 score
print(f"AVG F1 on 10000 random samples: {sum(f1_scores) / len(f1_scores)}")
print(f"AVG Accuracy on 10000 random samples: {sum(accuracy_scores) / len(accuracy_scores)}")

AVG F1 on 10000 random samples: 0.20544658565482288
AVG Accuracy on 10000 random samples: 0.1779285714285715


In [13]:
test_data = pd.read_csv(DATA_PATH+"articles_testing.tsv", sep='\t')
test_data

,content
0,"Vào tối 22/10, sau đêm bán kết của cuộc thi Ho..."
1,"Tân Hoa Xã đưa tin rằng vào ngày 9/10, phát ng..."
2,Chuyến thăm của Lloyd Austin diễn ra chỉ vài n...
3,"Trong dự thảo Luật Nhà giáo, Bộ GD&ĐT đã đưa r..."
4,"Trong buổi gặp gỡ, Chủ tịch Hà Thị Nga đã bày ..."
...,...
43898,"Giá vàng tiếp tục tăng phiên thứ hai, đạt đỉnh..."
43899,Để tỏ lòng tri ân với lịch sử và những người c...
43900,MangoTV vừa công bố danh sách phim có lượt xem...
43901,Để hoạt động đấu giá khoáng sản minh bạch và h...


In [14]:
def get_final_result(text):
	result_set = process_long_text(text, tokenizer)
	return ','.join(result_set)

In [15]:
print(get_final_result(longest_text))

Cực,Bắc Tông,Diệu Quang,Đế,tu viện Chơn Như,kinh Duy Ma C,Duy Ma Cất,Thủy,Đại,Thế Tôn,Đại Ca Diếp,Mục Kiền Liên,giáo,Châu,TL,Phật Giáo Nguyên Thủy,Lạc,Thông,Niết Bàn,kinh Duy Ma,Thường Chiếu,Phật học,Thích Ca,Phật giáo Nguyên,Nhà xuất bản Tôn giáo,Nam Tông,Ca Diếp,Ấn Độ,Đề Bà Đạt Đa,Pháp Loa,Phật,TPHCM,Cực Lạc,Đế Thích,Tông,Ng,Tây,Phật giáo Nguyên Thủy,Thiền Đông Độ,A Nan,Nguyễn Hòa,Bà La Môn,Thông Lạc,Tuệ Hạnh,Minh Hoàng,Duy Ma,Độ,Võ Đế,Duy Ma C,Anan,giáo Nguyên,Hoa Sen,Nan,Thiên,Huệ Năng,Viễn,đạo Phật,Trung Hoa,Bồ Đề Đạt Ma,Đường về,Phật giáo,núi Linh Thú,Duy Ma Cật,Huyền Quang,Phật Thích Ca,Thư Viện Hoa Sen,Việt Nam,giáo Nguyên Thủy


In [16]:
test_data['tags'] = test_data['content'].apply(get_final_result)

# save the result
test_data.to_csv(DATA_PATH+"nhom1_ner.tsv", sep='\t', index=False)

In [17]:
test_data

,content,tags
0,"Vào tối 22/10, sau đêm bán kết của cuộc thi Ho...","Lê Võ Quế Anh,Thái Lan,Nguyễn Minh Công,Việt N..."
1,"Tân Hoa Xã đưa tin rằng vào ngày 9/10, phát ng...","Anh,Sergei Naryshkin,Nga,Thụy Điển,Đan Mạch,No..."
2,Chuyến thăm của Lloyd Austin diễn ra chỉ vài n...,"miền Đông Ukraine,Donald Trump,Lầu Năm góc,Mát..."
3,"Trong dự thảo Luật Nhà giáo, Bộ GD&ĐT đã đưa r...","Bộ GD,Phan Thị Hằng Hải,ĐT,Nguyễn Thị Minh,Ngu..."
4,"Trong buổi gặp gỡ, Chủ tịch Hà Thị Nga đã bày ...","Orlando Nicolas Hernandez Guillen,Hà Thị Nga,H..."
...,...,...
43898,"Giá vàng tiếp tục tăng phiên thứ hai, đạt đỉnh...","SJC,Doji,TD Securities,Mỹ,Donald Trump,Nga Ukr..."
43899,Để tỏ lòng tri ân với lịch sử và những người c...,"Nguyễn Hoàng Đạo,Cà Mau,Bắc,Ban Tuyên giáo Tỉn..."
43900,MangoTV vừa công bố danh sách phim có lượt xem...,"Trương Tân Thành,Bạch Lộc,IQ,Tencent,Tống Thiế..."
43901,Để hoạt động đấu giá khoáng sản minh bạch và h...,"Bộ Tài nguyên và Môi trường,Hà Nam,Quảng Nam,H..."
